In [1]:
import os
import torch
import cv2
import numpy as np
from torchvision import transforms, utils, models
import torch.nn as nn
from tqdm import tqdm
from utils.data_process import preprocess_img, postprocess_img
from PIL import Image

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

flag = 0 # 0 for TranSalNet_Dense, 1 for TranSalNet_Res

↑↑↑  Set **flag=1** to load *TranSalNet_Dense*,set **flag=0** to load *TranSalNet_Res*. <br>
<br>
↓↓↓  Load the model and pre-trained parameters.<br>

In [4]:
from TranSalNet_Res import TranSalNet
model = TranSalNet()
model.load_state_dict(torch.load(r'pretrained_models/TranSalNet_Res.pth', map_location=torch.device('cpu')))

model = model.to(device) 
model.eval()

TranSalNet(
  (encoder): _Encoder(
    (encoder): ModuleList(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
      

↓↓↓ Get the test image, feed it into the model, and get a result.

In [7]:
test_img = r'./example/BenchmarkIMAGES/i33.jpg' 

img = preprocess_img(test_img) # padding and resizing input image into 384x288
img = np.array(img)/255.
img = np.expand_dims(np.transpose(img,(2,0,1)),axis=0)
img = torch.from_numpy(img)
img = img.type(torch.FloatTensor).to(device)
pred_saliency = model(img)
toPIL = transforms.ToPILImage()
pic = toPIL(pred_saliency.squeeze())

pred_saliency = postprocess_img(pic, test_img) # restore the image to its original size as the result

cv2.imwrite(r'example/sticker.png', pred_saliency, [int(cv2.IMWRITE_JPEG_QUALITY), 100]) # save the result
print('Finished, check the result at: {}'.format(r'example/result.png'))

(288, 384, 3)
Finished, check the result at: example/result.png


In [21]:
test_img = r'./example/BenchmarkIMAGES/i33.jpg' 

img = preprocess_img(test_img) # padding and resizing input image into 384x288


cv2.imwrite(r'example/result.png', img, [int(cv2.IMWRITE_JPEG_QUALITY), 100]) # save the result
print('Finished, check the result at: {}'.format(r'example/result.png'))

Finished, check the result at: example/result.png


In [54]:
from platform import python_version
python_version()

'3.9.12'